In [1]:
//Australia having very distinct 8 climate zones,no single model will work for all australia 
// Even within a state  there are multiple climate zones(from alpine to high humid) 
//Different models need to be trained for different locations
//In this approach,separate models will be created for different locations and saved in the directory
//for the prediction particular location model will be loaded and prediction will be done
//Data used was weatherAUS_YN.csv(before imputing the  null values) as it was  observed  that 
//dropping null values are more effective than imputing (In our scenario imputing means blunting the features i.e.  generalising the data)
//Wind direction (categorical columns) not tested or used in the modelling as too much of encoding required and crashing the docker
//correlation matrix(wrangling file)& trial & Error methods were used to select the features)

Intitializing Scala interpreter ...

Spark Web UI available at http://57b01b7ba74d:4040
SparkContext available as 'sc' (version = 2.4.5, master = local[*], app id = local-1591585066842)
SparkSession available as 'spark'


2020-06-08 02:57:42,105 WARN  [main] util.NativeCodeLoader (NativeCodeLoader.java:<clinit>(62)) - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Import required libraries

In [40]:
import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator  
import org.apache.spark.mllib.evaluation.MulticlassMetrics  
import org.apache.spark.mllib.evaluation.BinaryClassificationMetrics  
import org.apache.spark.ml.classification.LogisticRegression  
import org.apache.spark.ml.tuning.{ParamGridBuilder, TrainValidationSplit, CrossValidator,CrossValidatorModel}  
import org.apache.spark.ml.feature.{VectorAssembler, StringIndexer, OneHotEncoderEstimator,VectorIndexer}  
import org.apache.spark.ml.linalg.Vectors  
import org.apache.spark.ml.{Pipeline,PipelineModel}  
import org.apache.log4j._  
Logger.getLogger("org").setLevel(Level.ERROR)  

import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator
import org.apache.spark.mllib.evaluation.MulticlassMetrics
import org.apache.spark.mllib.evaluation.BinaryClassificationMetrics
import org.apache.spark.ml.classification.LogisticRegression
import org.apache.spark.ml.tuning.{ParamGridBuilder, TrainValidationSplit, CrossValidator, CrossValidatorModel}
import org.apache.spark.ml.feature.{VectorAssembler, StringIndexer, OneHotEncoderEstimator, VectorIndexer}
import org.apache.spark.ml.linalg.Vectors
import org.apache.spark.ml.{Pipeline, PipelineModel}
import org.apache.log4j._


# Starting spark session

In [3]:
import org.apache.spark.sql.SparkSession
val spark = SparkSession.builder().getOrCreate()

import org.apache.spark.sql.SparkSession
spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@465cd5d4


# Importing the file

In [4]:
val data = spark.read.option("header","true").
    option("inferSchema","true").
    format("csv").
    load("/home/weatherAUS_YN.csv")

data: org.apache.spark.sql.DataFrame = [Date: timestamp, Location: string ... 22 more fields]


In [5]:
// month is derived from the date filed to include the seasonal pattern to the model

In [6]:
val df= data.withColumn("Month",month(data("Date")))

df: org.apache.spark.sql.DataFrame = [Date: timestamp, Location: string ... 23 more fields]


#  Selecting required fields with right data type

In [7]:
//based on correlation matrix 

In [8]:
val df1 = df.selectExpr("Date","Location",
                        //"cast(MinTemp as double) MinTemp",
                        "cast (MaxTemp as double) MaxTemp",
                       // "cast (Rainfall as double) Rainfall",
                       // "cast (Evaporation as double) Evaporation",
                        // "cast (Sunshine as double) Sunshine",
                        //"WindGustDir",
                        // "cast (WindGustSpeed as double) WindGustSpeed",
                        //"WindDir9am","WindDir3pm",
                         //"cast (WindSpeed9am as double) WindSpeed9am",
                         //"cast (WindSpeed3pm as double) WindSpeed3pm",
                         //"cast (Humidity9am as double) Humidity9am",
                          "cast (Humidity3pm as double) Humidity3pm", 
                         //"cast (Pressure9am as double) Pressure9am",
                         "cast (Pressure3pm as double) Pressure3pm",
                         //"cast (Cloud9am as double) Cloud9am",
                         //"cast (Cloud3pm as double) Cloud3pm",
                        "cast (Temp3pm as double) Temp3pm",
                        "cast (Temp9am as double) Temp9am",
                         "cast (RainToday as integer) RainToday",
                         "cast (Month as String) Month",
                         "cast (RainTomorrow as integer) RainTomorrow"
                      
                                                  )
                        

df1: org.apache.spark.sql.DataFrame = [Date: timestamp, Location: string ... 8 more fields]


In [9]:
df1.columns

res4: Array[String] = Array(Date, Location, MaxTemp, Humidity3pm, Pressure3pm, Temp3pm, Temp9am, RainToday, Month, RainTomorrow)


# Filtering and forming a particular location data

In [10]:
// different models need to be created for different locations

In [11]:
val LocData=df1.filter($"Location"==="Perth")toDF()

LocData: org.apache.spark.sql.DataFrame = [Date: timestamp, Location: string ... 8 more fields]


In [12]:
// to see the no of records we have

In [13]:
LocData.count()

res7: Long = 3193


# Dropping the null values

In [14]:
val Loc =LocData.na.drop()

Loc: org.apache.spark.sql.DataFrame = [Date: timestamp, Location: string ... 8 more fields]


In [15]:
// checking the no of records after dropping null values

In [16]:
Loc.count()

res9: Long = 3184


# Setting the label column(Rain tomorrow)

In [17]:
val df2 = (Loc.select(Loc("RainTomorrow").as("label"), 
           $"Location",
           $"Date",
          //$"MinTemp",
          $"MaxTemp", 
         $"Month",
         $"Raintoday",
         //$"WindDir3pm",
         //$"WindSpeed9am",
         // $"WindSpeed3pm", 
         //$"Humidity9am",
          $"Humidity3pm",
          //$"Pressure9am",
          //$"Cloud3pm", 
          $"Pressure3pm", 
           $"Temp9am", 
          $"Temp3pm"))

df2: org.apache.spark.sql.DataFrame = [label: int, Location: string ... 8 more fields]


# Indexing the month column(categorical)-To capture the seasonal pattern

In [18]:
// Month  which is a categorical variable need  to be indexed and encoded to be used in logistic regression

In [19]:
val Indexer=new StringIndexer().setInputCol("Month").setOutputCol("MonIn")


Indexer: org.apache.spark.ml.feature.StringIndexer = strIdx_4128594a2cfc


# Encoding the MonIn( column

In [20]:
val Encoder=new OneHotEncoderEstimator().setInputCols(Array("MonIn")).setOutputCols(Array("MonEn"))


Encoder: org.apache.spark.ml.feature.OneHotEncoderEstimator = oneHotEncoder_1a50c20fe27d


# Setting the assembler

In [21]:
//setting the assembler with required fields for the regression model

In [22]:
 val assembler=(new VectorAssembler().setInputCols(Array(//"MinTemp",
                                                        "Raintoday",
                                                         "MaxTemp",
                                                        "Humidity3pm",
                                                         "Temp3pm",
                                                         "MonEn",
                                                        //"Humidity9am",
                                                        //"WindSpeed3pm",
                                                        "Pressure3pm"
                                                        )).setOutputCol("features"))

assembler: org.apache.spark.ml.feature.VectorAssembler = vecAssembler_ded2ea5a68d0


# Splitting the data into test and train sets

In [23]:
val Array(training,test)=df2.randomSplit(Array(0.7,0.3),seed=999)

training: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [label: int, Location: string ... 8 more fields]
test: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [label: int, Location: string ... 8 more fields]


# Logistic regression

In [24]:
//Maximum iteration was set to 30,more iterations giving minor improvements in the accuracy but more runtime and docker failures
//Threshold was set at 0.50 for the maximum accuracy(after trying multiple values)
// Thereshold can be adjusted if other evaluation parameters are preferred

In [25]:
val lr=new LogisticRegression().setMaxIter(10)
       

lr: org.apache.spark.ml.classification.LogisticRegression = logreg_2b24d4f81c5d


# Setting the pipeline

In [26]:
val pipeline=new Pipeline().setStages(Array(Indexer,Encoder,assembler,lr))

pipeline: org.apache.spark.ml.Pipeline = pipeline_6e0fb8f90b93


# Cross validation hyper parameter tuning

In [28]:
val paramGrid = new ParamGridBuilder()
  .addGrid(lr.regParam, Array(0.1, 0.01))
  .build()

paramGrid: Array[org.apache.spark.ml.param.ParamMap] =
Array({
	logreg_2b24d4f81c5d-regParam: 0.1
}, {
	logreg_2b24d4f81c5d-regParam: 0.01
})


In [29]:
val cv = new CrossValidator()
  .setEstimator(pipeline)
  .setEvaluator(new BinaryClassificationEvaluator)
  .setEstimatorParamMaps(paramGrid)
  .setNumFolds(3)  

cv: org.apache.spark.ml.tuning.CrossValidator = cv_3225ea819447


In [30]:
val cvModel = cv.fit(training)

2020-06-08 03:03:37,457 WARN  [Thread-4] netlib.BLAS (BLAS.java:<clinit>(61)) - Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
2020-06-08 03:03:37,475 WARN  [Thread-4] netlib.BLAS (BLAS.java:<clinit>(61)) - Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS
2020-06-08 03:03:40,076 ERROR [Thread-4] optimize.LBFGS (Logger.scala:error(27)) - Failure! Resetting history: breeze.optimize.FirstOrderException: Line search failed
2020-06-08 03:03:41,981 ERROR [Thread-4] optimize.LBFGS (Logger.scala:error(27)) - Failure! Resetting history: breeze.optimize.FirstOrderException: Line search failed
2020-06-08 03:03:43,490 ERROR [Thread-4] optimize.LBFGS (Logger.scala:error(27)) - Failure! Resetting history: breeze.optimize.FirstOrderException: Line search failed
2020-06-08 03:03:44,960 ERROR [Thread-4] optimize.LBFGS (Logger.scala:error(27)) - Failure! Resetting history: breeze.optimize.FirstOrderException: Line search failed
2020-06-08 03:03:46

cvModel: org.apache.spark.ml.tuning.CrossValidatorModel = cv_3225ea819447


In [31]:
//val model1=pipeline.fit(training)

# transforming test data with model

In [32]:
val results =cvModel.transform(test)

results: org.apache.spark.sql.DataFrame = [label: int, Location: string ... 14 more fields]


# Model Evaluation

In [33]:
val predictions=results.select($"prediction",$"label").as[(Double,Double)].rdd

predictions: org.apache.spark.rdd.RDD[(Double, Double)] = MapPartitionsRDD[855] at rdd at <console>:38


In [34]:
val metrics= new MulticlassMetrics(predictions)

metrics: org.apache.spark.mllib.evaluation.MulticlassMetrics = org.apache.spark.mllib.evaluation.MulticlassMetrics@34048db0


In [35]:
println(metrics.confusionMatrix)

764.0  18.0  
141.0  49.0  


(1,1) no  pred no  rain
(2,2) yes pred yes rain
(1,2) no pred yes rain 
(2,1)  yes pred no rain 

In [36]:
metrics.accuracy

res15: Double = 0.8364197530864198


# Saving the model

In [37]:
cvModel.write.overwrite().save("/home/model_Perth")

# Saving the pipeline

In [38]:
pipeline.write.overwrite().save("/home/pipeline_Perth")

# Reloading the model

In [41]:
val modelLoaded=CrossValidatorModel.load("/home/model_Perth")

modelLoaded: org.apache.spark.ml.tuning.CrossValidatorModel = cv_3225ea819447


# Using the model

In [42]:
// loaded model is applied on dataframe to see the prediction vs label

In [43]:
val dfu= modelLoaded.transform(df2)

dfu: org.apache.spark.sql.DataFrame = [label: int, Location: string ... 14 more fields]


In [44]:
df2.show()

+-----+--------+-------------------+-------+-----+---------+-----------+-----------+-------+-------+
|label|Location|               Date|MaxTemp|Month|Raintoday|Humidity3pm|Pressure3pm|Temp9am|Temp3pm|
+-----+--------+-------------------+-------+-----+---------+-----------+-----------+-------+-------+
|    0|   Perth|2008-07-01 00:00:00|   18.8|    7|        0|       53.0|     1024.5|    8.5|   18.1|
|    0|   Perth|2008-07-02 00:00:00|   20.7|    7|        0|       39.0|     1019.0|   11.1|   19.7|
|    1|   Perth|2008-07-03 00:00:00|   19.9|    7|        0|       71.0|     1015.6|   12.1|   17.7|
|    1|   Perth|2008-07-04 00:00:00|   19.2|    7|        1|       73.0|     1018.4|   13.2|   17.7|
|    1|   Perth|2008-07-05 00:00:00|   16.4|    7|        1|       57.0|     1022.1|   15.9|   16.0|
|    0|   Perth|2008-07-06 00:00:00|   15.9|    7|        1|       41.0|     1029.6|    6.9|   15.5|
|    0|   Perth|2008-07-07 00:00:00|   18.3|    7|        0|       36.0|     1024.2|    8.7

In [45]:
val testev=dfu.select("Location","Date","RawPrediction","probability","label","prediction").toDF()

testev: org.apache.spark.sql.DataFrame = [Location: string, Date: timestamp ... 4 more fields]


In [46]:
testev.show()

+--------+-------------------+--------------------+--------------------+-----+----------+
|Location|               Date|       RawPrediction|         probability|label|prediction|
+--------+-------------------+--------------------+--------------------+-----+----------+
|   Perth|2008-07-01 00:00:00|[0.73385840861598...|[0.67565140205561...|    0|       0.0|
|   Perth|2008-07-02 00:00:00|[1.22838640216562...|[0.77353603213384...|    0|       0.0|
|   Perth|2008-07-03 00:00:00|[0.32639785558474...|[0.58088266236470...|    1|       0.0|
|   Perth|2008-07-04 00:00:00|[-0.4721342546714...|[0.38411121870937...|    1|       1.0|
|   Perth|2008-07-05 00:00:00|[-0.2799448204272...|[0.43046730412009...|    1|       1.0|
|   Perth|2008-07-06 00:00:00|[0.06049107625464...|[0.51511815935146...|    0|       0.0|
|   Perth|2008-07-07 00:00:00|[1.11621338780469...|[0.75328566344203...|    0|       0.0|
|   Perth|2008-07-08 00:00:00|[1.12358031850862...|[0.75465222469232...|    1|       0.0|
|   Perth|